In [1]:
%load_ext autoreload
%autoreload 2

import os,sys,inspect
current_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parent_dir = os.path.dirname(os.path.dirname(os.path.dirname(current_dir)))
sys.path.insert(0, os.path.join(parent_dir))

In [2]:
import pickle
import pandas as pd

from sklearn.linear_model import LinearRegression
from catboost import CatBoostRegressor

from coord2vec.Noam_Adir.pipeline.preprocess import generic_clean_col, ALL_MANHATAN_FILTER_FUNCS_LIST
from coord2vec.Noam_Adir.pipeline.base_pipeline import extract_geographical_features, train_models, plot_scores


/data/home/morpheus/coord2vec_noam/coord2vec/common/parallel/multiproc_util.py:8: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
# features_bundle = karka_bundle_features
# use_full_dataset = True
# batch_size = 10000
# clean_funcs = [clean_floor_col, clean_constructionTime_col]
n_catboost_iter = 150

In [4]:
def load_manhatan_data_from_pickel(file_name="manhattan_house_prices.pkl"):
    manhatan_df = pickle.load( open( pickle_file_name, "rb" ) )
    manhatan_df["coord"] = manhatan_df.apply(lambda row: tuple(row[['lon', 'lat']].values), axis=1)
    return manhatan_df

In [5]:
def get_non_repeating_coords(full_df:pd.DataFrame):
    df = full_df.copy()
    if not "coord_id" in df.columns:
        coord_2coordid = {coord:i for i,coord in enumerate(manhatan_df["coord"].unique())}
        df["coord_id"] = manhatan_df.apply(lambda row: coord_2coordid[row["coord"]], axis=1)
    apeared_coord_id = {}
    coord_ids = df["coord_id"]
    for inx in df.index:
        if coord_ids[inx] in apeared_coord_id:
            df = df.drop([inx])
        else:
            apeared_coord_id[coord_ids[inx]]=True
#     print("test sould be one:", len(set(apeared_coord_id.values())))
    return  df
    

In [6]:
pickle_file_name = "manhattan_house_prices.pkl"
manhatan_df = pickle.load( open( pickle_file_name, "rb" ) )
manhatan_df["coord"] = manhatan_df.apply(lambda row: tuple(row[['lon', 'lat']].values), axis=1)

manhatan_df = generic_clean_col(manhatan_df, ALL_MANHATAN_FILTER_FUNCS_LIST)
cleaned_features = manhatan_df[['sold','priceSqft', 'numBedrooms', 'numBathrooms', 'sqft', 'coord']]
all_features = extract_geographical_features(cleaned_features)

In [7]:
all_features

,sold,priceSqft,numBedrooms,numBathrooms,sqft,coord,coord_id,distance_to_major_road_100m,length_of_major_road_100m,area_of_nearest_major_road_100m,...,area_of_nearest_building_50m,distance_to_building_100m,number_of_building_100m,area_of_building_100m,area_of_nearest_building_100m,distance_to_building_250m,number_of_building_250m,area_of_building_250m,area_of_nearest_building_250m,area_of_building_0m
0,900000.0,1384,1,1.0,650,"(-73.986817, 40.736249)",0,22.872667,465.826034,75.006598,...,1397.884593,0.000000,80.0,2630.299381,1397.884593,0.000000,269.0,10678.102122,1397.884593,129.866647
1,825000.0,1000,1,1.0,825,"(-73.97482099999999, 40.791194)",1,27.728680,2373.860017,1734.348480,...,777.107354,0.000000,47.0,1658.078754,777.107354,0.000000,256.0,8644.938566,777.107354,72.195035
2,540000.0,947,1,1.0,570,"(-73.968053, 40.760779)",2,29.472751,1387.348456,72.088905,...,918.006024,0.000000,37.0,2144.205749,918.006024,0.000000,206.0,11855.526306,918.006024,85.284841
3,1700000.0,1446,2,2.0,1175,"(-73.989811, 40.73043)",3,40.134080,793.254435,347.829569,...,1393.991316,0.000000,33.0,2111.188242,1393.991316,0.000000,263.0,9860.480359,1393.991316,129.504953
4,825000.0,916,2,1.0,900,"(-73.972274, 40.795986)",4,29.690490,1694.095759,723.844180,...,558.380436,0.000000,36.0,1626.600631,558.380436,0.000000,190.0,8049.831443,558.380436,51.874808
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16002,850000.0,1000,1,1.0,850,"(-73.984585, 40.780136999999996)",2035,25.115671,2057.076778,1152.925813,...,708.219453,0.000000,48.0,1701.318135,708.219453,0.000000,268.0,7996.404018,708.219453,65.795193
16003,999999.0,1176,1,1.0,850,"(-73.97379000000001, 40.793639)",4376,46.423191,604.256062,319.116799,...,526.422910,1.557385,53.0,1571.557034,526.422910,1.557385,220.0,8414.254623,526.422910,0.000000
16004,1410000.0,1046,3,2.0,1350,"(-73.958597, 40.766527)",5488,100.000000,0.000000,0.000000,...,742.309093,0.000000,69.0,2032.903064,742.309093,0.000000,228.0,10673.276929,742.309093,68.962197
16005,565000.0,1412,1,1.0,400,"(-74.002088, 40.732318)",4242,100.000000,0.000000,0.000000,...,182.288808,0.000000,126.0,1995.697937,182.288808,0.000000,566.0,9402.313632,182.288808,16.935043


## train on all of the data

In [8]:
models = [LinearRegression(),
          CatBoostRegressor(iterations=n_catboost_iter, learning_rate=1, depth=3)]
y_col = "sold"
training_cache = train_models(models=models, all_features=all_features, y_col=y_col)

0:	learn: 935180.5842394	total: 47.8ms	remaining: 7.12s
1:	learn: 850718.6598603	total: 49.5ms	remaining: 3.66s
2:	learn: 728631.1383448	total: 50.9ms	remaining: 2.5s
3:	learn: 647546.7858772	total: 52.4ms	remaining: 1.91s
4:	learn: 594949.3574876	total: 53.8ms	remaining: 1.56s
5:	learn: 558475.4511966	total: 55.3ms	remaining: 1.33s
6:	learn: 537477.0884153	total: 56.8ms	remaining: 1.16s
7:	learn: 514914.7384024	total: 58.3ms	remaining: 1.03s
8:	learn: 502444.8474691	total: 59.8ms	remaining: 936ms
9:	learn: 488539.9998171	total: 61.2ms	remaining: 857ms
10:	learn: 474620.8748399	total: 63.2ms	remaining: 798ms
11:	learn: 466953.6777749	total: 64.6ms	remaining: 743ms
12:	learn: 450108.1113142	total: 66.1ms	remaining: 697ms
13:	learn: 415001.1700222	total: 67.5ms	remaining: 656ms
14:	learn: 404178.0299105	total: 69ms	remaining: 621ms
15:	learn: 390398.7607129	total: 70.4ms	remaining: 590ms
16:	learn: 380279.7462221	total: 71.9ms	remaining: 563ms
17:	learn: 362113.7074620	total: 73.4ms	rema

In [9]:
plot_scores(training_cache)

mean price -  1601529.207896052
MSE: linear regression - 2997222394013.8574, catboost - 42851727253.59256
RMSE: linear regression - 1731248.7961046665, catboost - 207006.58746424608


## train on none repeating coords

In [10]:
print("number of columns:", len(manhatan_df.columns))
print("number of rows:", len(manhatan_df))
print("number of unique coord:", len(manhatan_df["coord"].unique()))

number of columns: 16
number of rows: 16007
number of unique coord: 5489


In [11]:
def statistics_of_repeating_coords(df, y_name):
    if not "coord_id" in df.columns:
        coord_2coordid = {coord:i for i,coord in enumerate(manhatan_df["coord"].unique())}
        manhatan_df["coord_id"] = manhatan_df.apply(lambda row: coord_2coordid[row["coord"]], axis=1)
    coord_y = df.loc[:, ["coord_id", y_name]]
    coord_statisics = coord_y.groupby(["coord_id"]).count()
    coord_statisics["std"] = coord_y.groupby(["coord_id"]).std()
    coord_statisics["mean"] = coord_y.groupby(["coord_id"]).mean()
    mean_std = coord_statisics.loc[coord_statisics[y_name]!=1, "std"].mean()
    df_mean = df[y_name].mean()
    print("mean std in price of appartment with the same coord:", mean_std)
    print("mean price in the dataset for comparison:", df_mean)
    coord_statisics = coord_statisics.rename(columns={y_name: "count"})
    return coord_statisics

In [12]:
statistics_of_repeating_coords(manhatan_df, "sold")

mean std in price of appartment with the same coord: 450228.5566634896
mean price in the dataset for comparison: 1600879.1584931593


,count,std,mean
coord_id,,,
0,11,189939.543611,8.558955e+05
1,1,NaN,8.250000e+05
2,19,190629.002690,5.775000e+05
3,17,671243.382704,1.273959e+06
4,1,NaN,8.250000e+05
...,...,...,...
5484,1,NaN,2.000000e+06
5485,1,NaN,1.090000e+06
5486,1,NaN,9.292880e+05


In [13]:
new_all_features = get_non_repeating_coords(all_features)
print("new number of rows", len(new_all_features))
training_cache = train_models(models=models, all_features=new_all_features, y_col=y_col)

new number of rows 16007
0:	learn: 1255334.6388004	total: 2.78ms	remaining: 414ms
1:	learn: 1005578.0315434	total: 5.06ms	remaining: 374ms
2:	learn: 870765.2740281	total: 7.69ms	remaining: 377ms
3:	learn: 788591.1513909	total: 9.74ms	remaining: 356ms
4:	learn: 756977.1296680	total: 11.9ms	remaining: 344ms
5:	learn: 718265.9404902	total: 14.1ms	remaining: 338ms
6:	learn: 670865.7783920	total: 16.3ms	remaining: 333ms
7:	learn: 655282.0854585	total: 18.3ms	remaining: 325ms
8:	learn: 604825.6838678	total: 20.8ms	remaining: 327ms
9:	learn: 564179.6834042	total: 23.5ms	remaining: 329ms
10:	learn: 513334.2465657	total: 28.1ms	remaining: 355ms
11:	learn: 492320.9262754	total: 32.2ms	remaining: 371ms
12:	learn: 471915.8232819	total: 36.1ms	remaining: 380ms
13:	learn: 462981.0228545	total: 40.9ms	remaining: 397ms
14:	learn: 447134.6041428	total: 45.5ms	remaining: 409ms
15:	learn: 436508.1664253	total: 49.5ms	remaining: 415ms
16:	learn: 426484.9762910	total: 53.7ms	remaining: 420ms
17:	learn: 414

In [14]:
plot_scores(training_cache)

mean price -  1804435.7392571012
MSE: linear regression - 3943043529011.7495, catboost - 102430448785.55797
RMSE: linear regression - 1985709.8300133757, catboost - 320047.57269124535


In [15]:
all_features

,sold,priceSqft,numBedrooms,numBathrooms,sqft,coord,coord_id,distance_to_major_road_100m,length_of_major_road_100m,area_of_nearest_major_road_100m,...,area_of_nearest_building_50m,distance_to_building_100m,number_of_building_100m,area_of_building_100m,area_of_nearest_building_100m,distance_to_building_250m,number_of_building_250m,area_of_building_250m,area_of_nearest_building_250m,area_of_building_0m
0,900000.0,1384,1,1.0,650,"(-73.986817, 40.736249)",0,22.872667,465.826034,75.006598,...,1397.884593,0.000000,80.0,2630.299381,1397.884593,0.000000,269.0,10678.102122,1397.884593,129.866647
1,825000.0,1000,1,1.0,825,"(-73.97482099999999, 40.791194)",1,27.728680,2373.860017,1734.348480,...,777.107354,0.000000,47.0,1658.078754,777.107354,0.000000,256.0,8644.938566,777.107354,72.195035
2,540000.0,947,1,1.0,570,"(-73.968053, 40.760779)",2,29.472751,1387.348456,72.088905,...,918.006024,0.000000,37.0,2144.205749,918.006024,0.000000,206.0,11855.526306,918.006024,85.284841
3,1700000.0,1446,2,2.0,1175,"(-73.989811, 40.73043)",3,40.134080,793.254435,347.829569,...,1393.991316,0.000000,33.0,2111.188242,1393.991316,0.000000,263.0,9860.480359,1393.991316,129.504953
4,825000.0,916,2,1.0,900,"(-73.972274, 40.795986)",4,29.690490,1694.095759,723.844180,...,558.380436,0.000000,36.0,1626.600631,558.380436,0.000000,190.0,8049.831443,558.380436,51.874808
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16002,850000.0,1000,1,1.0,850,"(-73.984585, 40.780136999999996)",2035,25.115671,2057.076778,1152.925813,...,708.219453,0.000000,48.0,1701.318135,708.219453,0.000000,268.0,7996.404018,708.219453,65.795193
16003,999999.0,1176,1,1.0,850,"(-73.97379000000001, 40.793639)",4376,46.423191,604.256062,319.116799,...,526.422910,1.557385,53.0,1571.557034,526.422910,1.557385,220.0,8414.254623,526.422910,0.000000
16004,1410000.0,1046,3,2.0,1350,"(-73.958597, 40.766527)",5488,100.000000,0.000000,0.000000,...,742.309093,0.000000,69.0,2032.903064,742.309093,0.000000,228.0,10673.276929,742.309093,68.962197
16005,565000.0,1412,1,1.0,400,"(-74.002088, 40.732318)",4242,100.000000,0.000000,0.000000,...,182.288808,0.000000,126.0,1995.697937,182.288808,0.000000,566.0,9402.313632,182.288808,16.935043
